In [48]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np 
import glob
import datetime

In [49]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [57]:
(train_image,train_labels),(test_image,test_labels) = tf.keras.datasets.mnist.load_data()

In [58]:
test_image.shape

(10000, 28, 28)

In [59]:
train_image = tf.expand_dims(train_image,-1)
test_image = tf.expand_dims(test_image,-1)

In [60]:
train_image = tf.cast(train_image/255,tf.float32)
test_image = tf.cast(test_image/255,tf.float32)

In [61]:
train_labels = tf.cast(train_labels,tf.int64)
test_labels = tf.cast(test_labels,tf.int64)

In [62]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_image,test_labels))

In [63]:
test_dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [64]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [65]:
dataset = dataset.shuffle(60000).repeat().batch(32)

In [66]:
test_dataset = test_dataset.batch(32).repeat()

In [67]:
model = tf.keras.Sequential()

In [68]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),input_shape=(None,None,1)
                           ,activation='relu'),
    tf.keras.layers.Conv2D(16,(3,3),activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10,activation='softmax'),
    
])

In [69]:
model.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['acc'])

In [86]:
import os
log_dir = os.path.join('logs',
        datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))

In [87]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
                'logs',histogram_freq=1)

In [88]:
def lr_sche(epoch):
    learning_rate = 0.2
    if epoch>5:
        learning_rate = 0.02
    if epoch>10:
        learning_rate = 0.01
    if epoch>20:
        learning_rate = 0.005
    tf.summary.scalar('learning_rate',data = learning_rate,step = epoch)
    return learning_rate

In [89]:
lr_callback  = tf.keras.callbacks.LearningRateScheduler(lr_sche)

In [90]:
file_writer = tf.summary.create_file_writer(log_dir+'/lr')
file_writer.set_as_default()

In [91]:
steps_per_epoch = train_image.shape[0]//32
validation_steps = test_image.shape[0]//32

In [92]:
steps_per_epoch

1875

In [93]:
model.fit(dataset,epochs=10,
         steps_per_epoch=steps_per_epoch,
         validation_data=(test_dataset),
          validation_steps=validation_steps,
         callbacks=[tensorboard_callback,lr_callback])

Train for 1875 steps, validate for 312 steps
Epoch 1/10
1875/1875 [==============================] - 16s 9ms/step - loss: 2.3892 - acc: 0.1033 - val_loss: 2.3359 - val_acc: 0.1135
Epoch 2/10
1875/1875 [==============================] - 16s 8ms/step - loss: 2.3259 - acc: 0.1024 - val_loss: 2.3318 - val_acc: 0.0975
Epoch 3/10
1875/1875 [==============================] - 15s 8ms/step - loss: 2.3270 - acc: 0.1012 - val_loss: 2.3120 - val_acc: 0.1135
Epoch 4/10
1875/1875 [==============================] - 15s 8ms/step - loss: 2.3258 - acc: 0.1032 - val_loss: 2.3091 - val_acc: 0.0975
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 2.3250 - acc: 0.1023 - val_loss: 2.3170 - val_acc: 0.1135
Epoch 6/10
1875/1875 [==============================] - 15s 8ms/step - loss: 2.3254 - acc: 0.1027 - val_loss: 2.3222 - val_acc: 0.0958
Epoch 7/10
1875/1875 [==============================] - 15s 8ms/step - loss: 2.3038 - acc: 0.1080 - val_loss: 2.3060 - val_acc: 0.1029
Epoch 8/10

In [94]:
%load_ext tensorboard
%matplotlib inline

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [97]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 32684), started 0:38:35 ago. (Use '!kill 32684' to kill it.)

# 自定义训练中使用Tensorboard